In [15]:
!pip install -U sentence-transformers -q  #Install dependencies

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
import pandas as pd

In [17]:
#Read the csv file file
text_data=pd.read_csv('/kaggle/input/dn-text/DataNeuron_Text_Similarity.csv', encoding='utf-8')
text_data.head(3)

,text1,text2
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...


In [18]:
#Since both the text pairs are unrelated we can merge the sentence pairs to form a single set
text_1=pd.concat([text_data['text1'],text_data['text2']],axis=0)
text_1=pd.DataFrame(text_1).reset_index().drop('index', axis=1)
index_counts = text_1.duplicated().sum()
print('The no of duplicates combining in both text1 and text2 are:',index_counts)
print('   ')
print('Dropping duplicates.....')
text_1=text_1.drop_duplicates()
print('   ')
print('No of sentences after dropping duplicates:')
len(text_1)

The no of duplicates combining in both text1 and text2 are: 4572
   
Dropping duplicates.....
   
No of sentences after dropping duplicates:


1428

In [19]:
#training the model using sentence transformers except for two sentences kept for testing.
text_1=list(text_1[0])
train=text_1[:1426]
test=text_1[1426:1428]

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import util
from sentence_transformers import losses
from sentence_transformers.losses import ContrastiveTensionLoss
from sentence_transformers.readers import InputExample
from torch.utils.data import DataLoader


#Train the model
model = SentenceTransformer('all-MiniLM-L6-v2')
train_examples = train
train_dataloader = ContrastiveTensionDataLoader(train_examples, batch_size=3, pos_neg_ratio=3)
train_loss = losses.ContrastiveTensionLoss(model=model)

#Model fitting
model.fit(
    [(train_dataloader, train_loss)],
    epochs=10,
)



In [ ]:
#save the tuned model
model.save('/kaggle/working/model') #Saving the model to be 


In [21]:
#Defining the function for calculating semantic sentence similarity
model_path='/kaggle/working/model'
model=SentenceTransformer(model_path)
def similarity(text1,text2,model):
    embeddings1 = model.encode(text1, convert_to_tensor=True)
    embeddings2 = model.encode(text2, convert_to_tensor=True)
    cosine_similarities = util.cos_sim(embeddings1, embeddings2)
    return cosine_similarities[0]

text1 = test[0]
text2 = test[1]
similarity(text1,text2,model)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tensor([0.1293], device='cuda:0')